In [1]:
pip install ydata-profiling

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import json
from ydata_profiling import ProfileReport

In [3]:
df = pd.read_json('dataset_big_patent_v3.json')
profile = ProfileReport(df, title="Profiling Report")

In [4]:
profile.to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.35it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
profile.to_notebook_iframe()

In [6]:
with open('dataset_big_patent_v3.json') as f:
    data = json.load(f)

df = pd.json_normalize(data, meta=['anchor', "query", "positive", "negative",])

# Display the DataFrame
df

,anchor,query,positive,negative
0,RELATED APPLICATIONS This application claims t...,What are the key advantages and applications o...,The present technology introduces an innovativ...,The invention relates to the design and utilit...
1,RELATED APPLICATIONS This application claims t...,How does a magnetic energy harvester operate w...,The advanced energy accumulation equipment bei...,The invention relates to the design and utilit...
2,RELATED APPLICATIONS This application claims t...,How does an energy harvester operate without a...,The invention relates to the design and utilit...,The present technology introduces an innovativ...
3,BACKGROUND OF THE INVENTION I. Field of the In...,How can buffer blocks for ruminant animals be ...,The innovative technique pertains to mineral s...,The latest invention provides novel systems an...
4,RELATED APPLICATION The present application cl...,What advancements does the described patent pr...,The current text discusses a novel mechanical ...,The present invention addresses various improv...
...,...,...,...,...
494,CROSS-REFERENCE TO RELATED APPLICATION [0001] ...,What is the role of Onjisaponin B in the treat...,Delineation of the effect of a botanical enhan...,This invention pertains to a unique compound f...
495,CROSS-REFERENCE TO RELATED APPLICATION [0001] ...,What is the mechanism by which Onjisaponin B e...,This invention pertains to a unique compound f...,Delineation of the effect of a botanical enhan...
496,RELATED APPLICATIONS [0001] This application i...,What are the advantages of the improved dental...,The present innovations focus on an enhanced s...,[0001] This document covers the detailed aspec...
497,FIELD [0001] The invention refers to a ventric...,What are the benefits and mechanisms of a nove...,The technology relates to an innovation in hem...,The present technology involves an advancement...


In [7]:
df.isna().sum()

anchor      0
query       0
positive    0
negative    0
dtype: int64

In [8]:
from IPython.display import display, HTML

display(HTML(df.sample(n=10).to_html()))

In [9]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [11]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# text = "The present disclosure relates to a disposable loading unit (DLU) or single use loading unit (SULU) for use with a surgical device"

# tokens = tokenizer(text, 
#                    padding=True, # Ajoute du padding pour uniformiser la longueur
#                    truncation=True, # Tronque les séquences trop longues
#                    return_tensors="pt") # Retourne des tenseurs PyTorch

# print(tokens)

In [12]:
from sklearn.model_selection import train_test_split

test_size = 0.2
val_size = 0.125

train_df, test_df = train_test_split(df, 
# 80% train, 20% test                              test_size=test_size,
                                     random_state=39)
train_df, val_df = train_test_split(train_df, 
                                     test_size=val_size/(1-test_size),
                                     random_state=39)
# 70% train, 10% validation

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

def tokenize_function(examples):
    return tokenizer(examples["abstract"], truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")

In [14]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe, texts, tokenizer, max_length):
        self.texts = []
        for column in column_names:
            self.texts.extend(dataframe[column].astype(str).tolist())  # Ajoute le texte de chaque colonne à la liste
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(text,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=self.max_length,
                                   return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': encoding['input_ids'].flatten()  # Pour l'auto-encoding
        }

In [15]:
# Nom des colonnes à tokeniser
column_names = df.columns.tolist()

In [16]:
train_dataset = CustomDataset(train_df, tokenizer, 512, column_names)
val_dataset = CustomDataset(val_df, tokenizer, 512, column_names)
test_dataset = CustomDataset(test_df, tokenizer, 512, column_names)

In [17]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [18]:
# from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./results",          # Dossier pour sauvegarder les résultats
#     overwrite_output_dir=True,
#     num_train_epochs=3,              # Nombre d'epochs
#     per_device_train_batch_size=16,  # Taille du batch par GPU
#     per_device_eval_batch_size=64,   # Taille du batch pour l'évaluation
#     save_steps=1000,                # Fréquence de sauvegarde des checkpoints
#     save_total_limit=2,              # Nombre maximum de checkpoints à conserver
#     prediction_loss_only=True,       # Optimise seulement la loss de prédiction
#     eval_steps=500,                  # Fréquence d'évaluation
#     learning_rate=5e-5,             # Taux d'apprentissage
#     weight_decay=0.01,               # Décroissance du poids
# )

# # 5. Entraînement du modèle
# model = AutoModel.from_pretrained("BAAI/bge-m3")

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )

# trainer.train()

# # 6. Sauvegarde du modèle fine-tuned
# model.save_pretrained("./bge-m3-fine-tuned")
# tokenizer.save_pretrained("./bge-m3-fine-tuned")

# print("Entraînement terminé !")

In [19]:
# from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
# import torch
# from torch.utils.data import Dataset
# import pandas as pd

# # 1. Préparation des données
# class CustomDataset(Dataset):
#     def __init__(self, texts, tokenizer, max_length):
#         self.texts = texts
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = str(self.texts[idx])
#         encoding = self.tokenizer(text,
#                                    truncation=True,
#                                    padding='max_length',
#                                    max_length=self.max_length,
#                                    return_tensors='pt')
#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten()
#         }

# # 2. Chargement du modèle et du tokenizer
# model_name = "BAAI/bge-m3"  # Remplace par le nom exact du modèle
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # 3. Préparation du dataset
# # Charge tes données à partir d'un fichier (CSV, JSON, etc.)
# data = pd.read_json('dataset_big_patent_v3.json')
# data = data.dropna(subset=['abstract'])
# texts = data['abstract'].tolist()  # Remplace 'text_column' par le nom de ta colonne de texte
# max_length = 512  # Adapte la longueur maximale des séquences

# dataset = CustomDataset(texts, tokenizer, max_length)

# # 4. Configuration de l'entraînement
# training_args = TrainingArguments(
#     output_dir="./results",          # Dossier pour sauvegarder les résultats
#     overwrite_output_dir=True,
#     num_train_epochs=3,              # Nombre d'epochs
#     per_device_train_batch_size=16,  # Taille du batch par GPU
#     save_steps=1000,                # Fréquence de sauvegarde des checkpoints
#     save_total_limit=2,              # Nombre maximum de checkpoints à conserver
#     prediction_loss_only=True,       # Optimise seulement la loss de prédiction
#     learning_rate=5e-5,             # Taux d'apprentissage
#     weight_decay=0.01,               # Décroissance du poids
# )

# # 5. Entraînement du modèle
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
# )

# trainer.train()

# # 6. Sauvegarde du modèle fine-tuned
# model.save_pretrained("./bge-m3-fine-tuned")
# tokenizer.save_pretrained("./bge-m3-fine-tuned")

# print("Entraînement terminé !")


In [20]:
# Nouvelle piste
# https://dassum.medium.com/fine-tune-large-language-model-llm-on-a-custom-dataset-with-qlora-fb60abdeba07

In [21]:
pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

Note: you may need to restart the kernel to use updated packages.


In [22]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np

In [23]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [24]:
# 1. Loading datatset
df = pd.read_json('dataset_big_patent_v3.json')

In [25]:
# 2. Create Bitsandbytes configuration
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [26]:
pip install -U bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [27]:
# 3. Loading the Pre-Trained model
from transformers import AutoTokenizer

model_name = "BAAI/bge-m3"

device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      token=True)



The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# 4. Tokenization with left padding

tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 5. Test the model with zero-shopt inferencing
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

# Choses à faire demain

1. jupyter notebook gpu acceleration
2. https://www.datacamp.com/fr/tutorial/fine-tuning-large-language-models
3. Savoir quelle colonne tokenizer
4. Supervised learning -> Q&A # https://towardsdatascience.com/fine-tuning-large-language-models-llms-23473d763b91/

In [32]:
!pip install tensorflow[and-cuda]

  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.0.1-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
  Using cached nvidia_cublas_cu12-12.5.3.2-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.5.82-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached nvidia_cuda_nvcc_cu12-12.5.82-py3-none-win

ERROR: Cannot install tensorflow[and-cuda]==2.16.1, tensorflow[and-cuda]==2.16.2, tensorflow[and-cuda]==2.17.0, tensorflow[and-cuda]==2.17.1, tensorflow[and-cuda]==2.18.0, tensorflow[and-cuda]==2.18.1 and tensorflow[and-cuda]==2.19.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [35]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

ModuleNotFoundError: No module named 'tensorflow'